In [12]:
import pickle

import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz
%matplotlib inline

In [66]:
# initialize graph
G = nx.DiGraph()

# load dictionary
circuit_path = "../simple_circuit.pkl"
circuit_figure_path = circuit_path.split(".pkl")[0] + ".png"
with open(circuit_path, "rb") as circuit_pickle:
    circuit = pickle.load(circuit_pickle)

In [67]:
G.graph['node'] = {'shape': 'box', 'style': 'rounded'}
G.graph['graph'] = {'rankdir': 'BT'}

for parent in circuit:
    if parent not in G.nodes:
        if "mlp" in parent:
            color = "#e5ad76"
        elif "attn" in parent:
            color = "#5baf7e"
        elif "resid" in parent:
            color = "#88a7cd"
        else:
            color = "white"

        if parent != "y":
            layer, feat_idx, component = parent.split("_")
            nodename = f"{component}_{layer}/{feat_idx}"
        else:
            nodename = "y"
        G.add_node(parent, label=nodename, style="rounded,filled", fillcolor=color)
    for child in circuit[parent]:
        child_name, child_effect = child
        if child_name not in G.nodes:
            if "mlp" in child_name:
                color = "#e5ad76"
            elif "attn" in child_name:
                color = "#5baf7e"
            elif "resid" in child_name:
                color = "#88a7cd"
            else:
                color = "white"

            layer, feat_idx, component = child_name.split("_")
            nodename = f"{component}_{layer}/{feat_idx}"
            G.add_node(child_name, label=nodename, style="rounded,filled", fillcolor=color)
        G.add_edge(child_name, parent)

A = to_agraph(G)
A.layout('dot')
A.draw(circuit_figure_path)